# Notebook 1: Loading an image from Listmode data.

In this notebook we will use Python functions to:
- load and inspect the data
- generate a visualizable image
- save your image for use or analysis elsewhere

## Import packages

First, move your working directory upwards one so you can access source code in iqid/. This step is necessary because I haven't set up iqid as an installable package (i.e. via pip or conda), reflecting the fact that it's pretty informal and (permannently) in development.

In [1]:
cd ..

C:\Users\Robin\Documents\Cal\iQID\git_branches\iqid-alphas


In [2]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from skimage import io

%load_ext autoreload
%autoreload 2
from iqid import helper as iq # helper functions such as plotting parameters and other things
from iqid import process_object as po # class to handle listmode data is contained here.

pltmap = iq.set_plot_parms() # set default plot settings (feel free to change this)

C:\Users\Robin\anaconda3\envs\beq\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


#  Generate image from listmode data
- The %%time command will make the code block tell you how much time it took to run.
- Larger data will take more time to import.

In [ ]:
%%time

# enter the path to your iQID data on your machine
datdir = r"C:\Users\Robin\Documents\Cal\iQID\git_branches\iqid-alphas\data_sample\File_Structure_Sample"

# identify the list-mode data file to be processed, in this case the latest one in the Listmode folder
fname = iq.natural_sort(glob.glob(os.path.join(datdir, "Listmode", "*Compressed_Processed_Listmode.dat")))[-1]

# use the iQID code to load the data and create an image
# since we're working with the Processed LM file, use "processed_lm"
# we're using the minimum cluster area filter for now (area threshold = 1)
cdat = po.ClusterData(fname, ftype="processed_lm", c_area_thresh=1)
cim = cdat.image_from_big_listmode(event_fx=1)

# Display the image

In [ ]:
f, ax = plt.subplots(1,1, figsize=(8,4))
im = ax.imshow(cim, cmap='inferno')
ax.axis('off')

cbar = iq.nice_colorbar(im)
cbar.set_label('Counts/Px')

plt.title('Unformatted image')
plt.tight_layout()
plt.show()
plt.close()

This image is *quantitative* but not *calibrated*. That means, the relative uptake in the different parts of the image are accurate with respect to each other. However, without further analysis, we can't know the absolute Activity (Bq or Ci) in the image.

For now though, we want to just check the data and see if it looks ok. It's a little hard to see because the uptake in the mets is so much higher than in the rest of the liver tissue (which is a great result). Let's generate a log-plot of the image so we can see the extent of the tissues.

In [ ]:
# add threshold below which to black out in log img, then apply to a copy of the img
small_value = np.min(cim[cim > 0])
im_display = np.copy(cim)
im_display[im_display==0] = small_value 

im = plt.imshow(im_display, cmap='inferno', norm=LogNorm())
plt.axis('off')
cbar = iq.nice_colorbar(im)
cbar.set_label('Counts/Px')
plt.title('Log-scale image')
plt.show()

A little better, but still hard to see. How about this - we'll just make the window extremely small.

In [ ]:
f, ax = plt.subplots(1,1, figsize=(8,4))
im = ax.imshow(cim, cmap='inferno', vmax=0.01*np.max(cim))
ax.axis('off')

cbar = iq.nice_colorbar(im)
cbar.set_label('Counts/Px')

plt.title('Read the note below!')
plt.tight_layout()
plt.show()
plt.close()

<div class="alert alert-block alert-warning">
<b>Warning!</b> Note that now that we've adjusted the window down, the colorbar is <b>no longer accurate.</b> We capped the maximum display at about 3 counts/px, which is much, much less than some pixels had (see the above image, which goes up to >300 counts/px.)
</div>

# Save the image
We can save the image in one of two ways.
1. Save the .tif image to preserve quantitative properties. Use this if you want to analyze in other software or continue to quantitative dosimetry.
2. Save the .png image of the figure we made. Use this if you want to produce a figure for a manuscript, poster, etc.

## 1. Save .tif for analysis

In [ ]:
# pick where you want to save the image and its name
# this defaults to the iQID data folder
io.imsave(os.path.join(datdir, "my_tif_image.tif"), cim, plugin='tifffile')

## 2. Save .png for figure
Showing the log-scale image as an example.

In [ ]:
# add threshold below which to black out in log img, then apply to a copy of the img
small_value = np.min(cim[cim > 0])
im_display = np.copy(cim)
im_display[im_display==0] = small_value 

im = plt.imshow(im_display, cmap='inferno', norm=LogNorm())
plt.axis('off')
cbar = iq.nice_colorbar(im)
cbar.set_label('Counts/Px')
plt.title('Log-scale image')
plt.tight_layout()

# here is the line to save the figure
# you can adjust the location and settings as desired
plt.savefig(os.path.join(datdir, "my_tif_image.png"), bbox_inches='tight', dpi=300)

# closes the figure without displaying it
plt.close()

# Things to check

Curious if your acquisition went well? Suspicious that something is wrong? Here are two things I like to check.

- Time histogram
- Missed images

## 1. Time histogram
We expect to see a nice exponential decay corresponding to the half-life of the isotope that you used. Let's look at a good example first (from the most recent data).

*note that if you run the next examples and come back to this one, you'll have to re-import the data.*

In [ ]:
# regardless of what time unit you decide to use, make sure it's consistent for all arrays
thalf = 9.9 * 24 # time 
tdata = cdat.t_s / 60 / 60 # timestamps of each recorded event in hours
t_binsize = 1 # 1-hour binsize -- adjust if needed
nbins = int(np.round((tdata[-1] - tdata[0])/t_binsize))

# generate histogram
n, bins, _ = plt.hist(tdata, bins=nbins, edgecolor='white')
binedges = 0.5 * (bins[1:] + bins[:-1])

# fit the histogram with an exponential decay
popt, pcov, param_std, res, chisq, chisqn = cdat.fitHist(
    binedges, n, func=po.exponential, p0=[1, thalf], tol=0.05)

# generate dummy array to plot the exponential
tdummy = np.linspace(tdata[0], tdata[-1], 5000)

# plot the components on top of the histogram
plt.errorbar(binedges, n, np.sqrt(n), linestyle='none', capsize=3) # errorbars
plt.plot(tdummy, po.exponential(tdummy, *popt), color='k')
plt.xlabel('Time (h)')
plt.ylabel('Counts / bin')
plt.title('Good time histogram with exp fit')
plt.show()

This histogram appears to be reasonably fit with the half-life of Ac-225 (to within 5% tolerance). There are no obvious spikes or missing data during the acquisition. Note that because 10 d is a long half-life, it will look fairly linear over a 24-h acquisition. Looks good to me!

What might a bad example look like and what would it tell you about the data?

In [ ]:
datdir = r"C:\Users\Robin\Documents\Cal\iQID\git_branches\iqid-alphas\data_sample\File_Structure_Sample_Bad"
fname = iq.natural_sort(glob.glob(os.path.join(datdir, "Listmode", "*Compressed_Processed_Listmode.dat")))[-1]
cdat = po.ClusterData(fname, ftype="processed_lm", c_area_thresh=1)
cim = cdat.image_from_big_listmode(event_fx=1)

In [ ]:
tdata = cdat.t_s / 60 / 60 # timestamps of each recorded event in hours
nbins = int(np.round((tdata[-1] - tdata[0])/t_binsize))

# generate histogram
n, bins, _ = plt.hist(tdata, bins=nbins, edgecolor='white')
binedges = 0.5 * (bins[1:] + bins[:-1])

# fit the histogram with an exponential decay
popt, pcov, param_std, res, chisq, chisqn = cdat.fitHist(
    binedges, n, func=po.exponential, p0=[1, thalf], tol=0.05)

# generate dummy array to plot the exponential
tdummy = np.linspace(tdata[0], tdata[-1], 5000)

# plot the components on top of the histogram
plt.errorbar(binedges, n, np.sqrt(n), linestyle='none', capsize=3) # errorbars
plt.plot(tdummy, po.exponential(tdummy, *popt), color='k')
plt.xlabel('Time (h)')
plt.ylabel('Counts / bin')
plt.title('Bad time histogram with exp fit')
plt.show()

We can see that this histogram does not follow a nice exponential decay like the last one, instead showing a dip in signal from 3h-14h. One guess for why this might have happened is if the laptop had some kind of scheduled update or power setting that made it divert computational resources during that time. Data like this may still be quantitatively recoverable with some further analysis. Since there's still a lot of counts, the image itself should be fine. Let's see below:

In [ ]:
f, ax = plt.subplots(1,1, figsize=(8,4))
im = ax.imshow(cim, cmap='inferno', vmax=0.5*np.max(cim))
ax.axis('off')

cbar = iq.nice_colorbar(im)
cbar.set_label('Counts/Px')

plt.title('The image is fine,\nbut care needed for future analysis.')
plt.tight_layout()
plt.show()
plt.close()

## 2. Missed images

In the iQID header file, you can check the last line to see if any images (frames) were missed during the acquisition. Significant (>100) missed frames can indicate something went wrong. Sometimes, if the acquisition doesn't stop correctly, the missed image warning at the bottom of the header file will be missing. We can check manually using the Offsets Listmode file.

In [ ]:
# first: good example with no missed images
datdir = r"C:\Users\Robin\Documents\Cal\iQID\git_branches\iqid-alphas\data_sample\File_Structure_Sample"

# note, using the Offsets file instead
oname = glob.glob(os.path.join(datdir,'Listmode', '*Offsets_Full_Raw_Listmode.dat'))[-1]
odat = po.ClusterData(oname, ftype="offset_lm")
data = odat.load_cluster_data(dtype=np.int32)
f_of, t, m, n, px, kelv = odat.init_metadata(data)

# plot missed images over time
plt.plot(t * 1e-3 / 3600, m)
plt.xlabel('Time (h)')
plt.ylabel('Missed images')
plt.title('First data set has no missed images')
plt.show()

In [ ]:
# next: example with missed images
datdir = r"C:\Users\Robin\Documents\Cal\iQID\git_branches\iqid-alphas\data_sample\File_Structure_Sample_Missed"

# note, using the Offsets file instead
oname = glob.glob(os.path.join(datdir,'Listmode', '*Offsets_Full_Raw_Listmode.dat'))[-1]
odat = po.ClusterData(oname, ftype="offset_lm")
data = odat.load_cluster_data(dtype=np.int32)
f_of, t, m, n, px, kelv = odat.init_metadata(data)

# plot missed images over time
plt.plot(t * 1e-3 / 3600, m)
plt.xlabel('Time (h)')
plt.ylabel('Missed images')
plt.title('Constantly missed images = something is wrong')
plt.show()

In this data, we were having issues with the old iQID laptop as well as connection via USB. There will be some problems doing quantitative dosimetry with this.

In some datasets, there might just be one spike of missed images. This is usually recoverable but it's a case-by-case basis.

# Summary
This notebook showed the basics for loading and simple image construction of iQID listmode data. We also learned how to troubleshoot to identify a couple of the common issues that might result from device problems during acquisition. We haven't discussed how to resolve or correct for these issues yet, but knowing is half the battle!

In the next demo, we'll show how to perform a basic quantitative evaluation of a well-behaved data set.